In [17]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

import healpy as hp
from scipy.spatial import ConvexHull

import lsst.daf.butler as dafButler
from lsst.analysis.ap import apdb
from lsst.ap.association import AssociationTask, AssociationConfig
from lsst.dax.apdb import Apdb, ApdbCassandra, ApdbTables
import lsst.geom
from lsst.afw import image as afwImage

from mpl_toolkits.axes_grid1 import make_axes_locatable
from astropy.visualization import ZScaleInterval, SqrtStretch, ImageNormalize, ManualInterval, AsinhStretch, MinMaxInterval, LogStretch


import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

from typing import TYPE_CHECKING, cast

import astropy.units as u
from astropy.coordinates import SkyCoord

plt.set_loglevel('WARNING')

# Custom functions

In [18]:
def create_mag_errors(sciFlux, sciFluxErr):
    """Move flux into magnitudes and calculate the error on the magnitude
    
    Parameters
    ----------
    sciFlux : `float`
        Science flux
    sciFluxErr : `float`
        Science flux error

    Returns
    -------
    mag, magerr  : `float`, `float`
        Magnitude and magnitude error
    """
    
    mag = u.nJy.to(u.ABmag, sciFlux)
    upper_mag = u.nJy.to(u.ABmag, sciFlux+sciFluxErr)
    lower_mag = u.nJy.to(u.ABmag, sciFlux-sciFluxErr)
    magErr = -(upper_mag-lower_mag)/2
    
    return mag, magErr

def create_mag(sciFlux):
    """Move flux into magnitudes
    
    Parameters
    ----------
    sciFlux : `float`
        Science flux

    Returns
    -------
    mag  : `float`
        Magnitude
    """
    
    mag = u.nJy.to(u.ABmag, sciFlux)
    
    return mag

def degrees_to_radians(degrees):
    """
    Convert an angle from degrees to radians.

    Parameters:
    degrees (float): Angle in degrees.

    Returns:
    float: Angle in radians (unitless).
    """
    # Convert the input degrees to radians
    radians = (degrees * u.deg).to(u.rad)
    
    # Return the numerical value (without the unit)
    return radians.value

def radians_to_degrees(radians):
    """
    Convert an angle from radians to degrees.

    Parameters:
    radians (float): Angle in radians.

    Returns:
    float: Angle in degrees (unitless).
    """
    # Convert the input radians to degrees
    degrees = (radians * u.rad).to(u.deg)
    
    # Return the numerical value (without the unit)
    return degrees.value

def flux_to_magnitude(df_single_flux_src):
    """Magic function that converts flux to magnitude

    Args:
        df_single_flux_src (_type_): _description_

    Returns:
        _type_: _description_
    """
    
    # Define the zeropoints for each band
    zeropoints = {
        'u': 12.652350670009373,
        'g': 14.689449213373276,
        'r': 14.559501946792281,
        'i': 14.378976834902065,
        'z': 13.993052964496345,
        'y': 13.017367314857484
    }
    
    # Create an empty dictionary to hold magnitudes
    mag_dict = {}
    
    # Apply the transformation for each band
    for band in zeropoints.keys():
        # Make sure the flux column for the band exists in the dataframe
        if f'lsst_flux_{band}' in df_single_flux_src.columns:
            flux_col = f'lsst_flux_{band}'
            mag_col = f'mag_{band}'
            
            # Calculate magnitudes from flux: mag = zp - 2.5 * log10(flux)
            # Handling zero or negative flux values to avoid invalid log10 operations
            mag_dict[mag_col] = zeropoints[band] - 2.5 * np.log10(df_single_flux_src[flux_col].replace(0, np.nan))
    
    # Convert the dictionary to a DataFrame and concatenate with the original dataframe
    mag_df = pd.DataFrame(mag_dict)
    
    # Return the original dataframe with new magnitude columns
    return pd.concat([df_single_flux_src, mag_df], axis=1)


In [19]:
# at the moment comcam only in Cassandra
# from lsst.analysis.ap import apdbCassandra
# or4_apdb_Cassandra = apdbCassandra.DbCassandraQuery(config_uri='s3://embargo@rubin-summit-users/apdb_config/cassandra/pp_apdb_lsstcomcamsim_or4.py')
# comcam_apdb_Cassandra = apdbCassandra.DbCassandraQuery(config_uri='s3://embargo@rubin-summit-users/apdb_config/cassandra/pp_apdb_lsstcomcam.py')

# Import data

In [20]:
# Load collection
repo = "embargo_new"
collection = "LSSTComCam/runs/DRP/20241101_20241127/w_2024_48/DM-47841"
instrument = "LSSTComCam"

butler = dafButler.Butler(repo, collections=collection, instrument=instrument)

In [21]:
bands = ["u", "g", "r", "i", "z", "y"]

verbose = False

for band in bands:
    datasetRefs_dia = butler.query_datasets("goodSeeingDiff_differenceExp", where=f"band='{band}'")

    if verbose:
        print(f"\nDataset references for band '{band}':")
        for dr in datasetRefs_dia:
            print(dr)

    print(f"Found {len(datasetRefs_dia)} differenceExps for band '{band}'")

In [22]:
i = 50
data_id = datasetRefs_dia[i].dataId
print(data_id)

In [62]:
lc_refs = butler.query_datasets("diaSourceTable_tract")
obj_refs = butler.query_datasets("diaObjectTable_tract")
print(len(lc_refs))
print(len(obj_refs))

In [67]:
# List of tract indices to process
tract_indices = np.arange(0, 43)

# Initialize lists to hold diaObj and diaSource DataFrames
dia_Obj_list = []
diaSource_list = []

# Loop through the specified tracts and fetch the data
for idx in tract_indices:
    obj_ref = obj_refs[idx]  # Select the corresponding obj_ref
    dia_Obj = butler.get(obj_ref)  # Fetch diaObj for this tract
    diaSource = butler.get('diaSourceTable_tract', dataId=obj_ref.dataId)  # Fetch diaSource for this tract
    
    # Append the results to the lists
    dia_Obj_list.append(dia_Obj)
    diaSource_list.append(diaSource)

# Concatenate all diaObj and diaSource DataFrames
combined_dia_Obj = pd.concat(dia_Obj_list, ignore_index=False)
combined_diaSource = pd.concat(diaSource_list, ignore_index=False)

dia_Obj = combined_dia_Obj.copy()
diaSource = combined_diaSource.copy()

# Compute the histogram of nDiaSources
counts, bin_edges = np.histogram(combined_dia_Obj['nDiaSources'], bins=40, range=(0, 40))

# Take the logarithm (base 10) of the counts
log_counts = np.log10(counts + 1)  # Add 1 to avoid log(0)

# Plot the histogram
plt.bar(bin_edges[:-1], log_counts, width=np.diff(bin_edges), align='edge', edgecolor='black', alpha=0.7)
plt.xlabel('Number of diaSources per diaObject')
plt.ylabel('Log10(Number of diaObjects)')
plt.title('Log Number of diaObjects per Number of Sources per Object')

# Number of diaObjects per magnitude

In [30]:
output_string = (
    f"Number of unique visits: {len(np.unique(diaSource['visit']))}, "
    f"Number of unique diaObjectId: {len(np.unique(diaSource['diaObjectId']))}, "
    f"Total number of diaSource entries: {len(diaSource)}"
)

print(output_string)

In [31]:
"""
dia = butler.get("goodSeeingDiff_differenceExp", dataId=data_id)
template = butler.get("goodSeeingDiff_templateExp", dataId=data_id)
calexp = butler.get("calexp", dataId=data_id)
src = butler.get("src", dataId=data_id)
dia_src = butler.get("goodSeeingDiff_diaSrc", dataId=data_id)
dia_src_table = butler.get("goodSeeingDiff_diaSrcTable", dataId=data_id) 
"""

In [32]:
# Step 1: Calculate mean flux and mean SNR for each diaObjectId in each band
mean_flux_per_object_band = diaSource.groupby(['diaObjectId', 'band'])['scienceFlux'].mean().unstack(fill_value=0)
mean_snr_per_object_band = diaSource.groupby(['diaObjectId', 'band'])['snr'].mean().unstack(fill_value=-99)

# Rename columns dynamically for all bands
band_flux_columns = {band: f"{band}_band_flux_mean" for band in ['i', 'g', 'r', 'y', 'z', 'u']}
band_snr_columns = {band: f"{band}_band_snr_mean" for band in ['i', 'g', 'r', 'y', 'z', 'u']}
mean_flux_per_object_band = mean_flux_per_object_band.rename(columns=band_flux_columns).reset_index()
mean_snr_per_object_band = mean_snr_per_object_band.rename(columns=band_snr_columns).reset_index()

# Step 2: Merge mean flux and SNR values into dia_Obj_field
dia_Obj_field_updated = dia_Obj.merge(mean_flux_per_object_band, on='diaObjectId', how='left')
dia_Obj_field_updated = dia_Obj_field_updated.merge(mean_snr_per_object_band, on='diaObjectId', how='left')

# Step 3: Calculate magnitudes for all bands
for band in tqdm(['i', 'g', 'r', 'y', 'z', 'u']):
    flux_col = f"{band}_band_flux_mean"
    mag_col = f"{band}_band_mag"
    dia_Obj_field_updated[mag_col] = dia_Obj_field_updated[flux_col].apply(create_mag)

# Replace infinite values with NaN
dia_Obj_field_updated.replace([np.inf, -99, -np.inf], np.nan, inplace=True)

# Select and display the magnitude columns, SNR, and diaObjectId
output_columns = ['diaObjectId'] + \
                 [f"{band}_band_mag" for band in ['i', 'g', 'r', 'y', 'z', 'u']] + \
                 [f"{band}_band_snr_mean" for band in ['i', 'g', 'r', 'y', 'z', 'u']]

result = dia_Obj_field_updated[output_columns]

# Display the resulting DataFrame
result 

In [33]:
result

In [34]:
result.to_parquet('result.parquet', index=False)

print("DataFrame saved as 'result.parquet'.")

In [35]:
plt.figure(figsize=(12, 10))
plt.suptitle('Number of detected diaObjects, DRP, LSSTComCam/runs/DRP/20241101_20241127/w_2024_48/DM-47841')

plt.subplot(6, 1, 1)
plt.hist(result['u_band_mag'][~np.isnan(result['u_band_mag'])], color='blue', alpha=0.5, bins=np.arange(15,30,0.1), label='u band detected objects');
plt.xlim(15, 30)
plt.xlabel('Magnitude')
plt.legend();

plt.subplot(6, 1, 2)
plt.hist(result['g_band_mag'][~np.isnan(result['g_band_mag'])], color='g', alpha=0.5, bins=np.arange(15,30,0.1), label='g band detected objects');
plt.xlim(15, 30)
plt.xlabel('Magnitude')
plt.legend();

plt.subplot(6, 1, 3)
plt.hist(result['r_band_mag'][~np.isnan(result['r_band_mag'])], color='r', alpha=0.5, bins=np.arange(15,30,0.1), label='r band detected objects');
plt.xlim(15, 30)
plt.xlabel('Magnitude')
plt.legend();

plt.subplot(6, 1,4)
plt.hist(result['i_band_mag'][~np.isnan(result['i_band_mag'])], color='brown', alpha=0.5, bins=np.arange(15,30,0.1), label='i band detected objects');
plt.xlim(15, 30)
plt.xlabel('Magnitude')
plt.legend();

plt.subplot(6, 1,5)
plt.hist(result['z_band_mag'][~np.isnan(result['z_band_mag'])], color='darkorange', alpha=0.5, bins=np.arange(15,30,0.1), label='z band detected objects');
plt.xlim(15, 30)
plt.xlabel('Magnitude')
plt.legend();

plt.subplot(6, 1,6)
plt.hist(result['y_band_mag'][~np.isnan(result['y_band_mag'])], color='gray', alpha=0.5, bins=np.arange(15,30,0.1), label='y band detected objects');
plt.xlim(15, 30)
plt.xlabel('Magnitude')
plt.legend();

In [36]:

# Define colors for each band
band_colors = {
    'u': 'blue',
    'g': 'green',
    'r': 'red',
    'i': 'brown',
    'z': 'darkorange',
    'y': 'gray'
}

# Define the bands
bands = ['u', 'g', 'r', 'i', 'z', 'y']

# Create subplots with one figure per band
fig, axes = plt.subplots(len(bands), 1, figsize=(8, 18), sharex=True)
fig.suptitle('SNR Distribution Across Bands', fontsize=16)

for ax, band in zip(axes, bands):
    # Dynamically get the correct SNR column for the current band
    column_name = f'{band}_band_snr_mean'
    snr_values = result[column_name][~np.isnan(result[column_name])]
    
    # Plot the histogram for the current band
    ax.hist(
        snr_values, 
        color=band_colors[band], 
        alpha=0.7, 
        bins=np.arange(0, 30, 0.1), 
        label=f'{band} band detected objects'
    )
    ax.set_ylabel('Frequency')
    ax.set_title(f'{band} Band')
    ax.legend()

# Set common x-axis label
axes[-1].set_xlabel('SNR')

# Adjust layout for better spacing
plt.tight_layout(rect=[0, 0, 1, 0.97])  # Leave space for the suptitle
plt.show()

In [38]:
diaSource.columns

# Positive and negative sources

In [42]:
# Let us look at the flux distribution of the sources and see if it is reasonably symmetric
plt.hist(diaSource['psfFlux'],bins=np.arange(-20000,20000,200));

In [43]:
np.sum(diaSource[diaSource['psfFlux']<0]['shape_flag'])/len(diaSource[diaSource['psfFlux']<0]['shape_flag'])

In [44]:
single_obj = dia_Obj[dia_Obj['nDiaSources'] > 10].iloc[230]
single_obj

# MVW plot (i.e., shapes)

In [59]:
# Select a specific band, e.g., 'i'
selected_band = 'g'
src_subselection = diaSource.copy()

# Add 'diaSourceId' as a column
src_subselection['diaSourceId'] = src_subselection.index

# Filter for the selected band
# src_subselection = src_subselection[src_subselection['band'] == selected_band]

# Subset the relevant columns
src_subselection = src_subselection[['diaSourceId', 'ixx', 'iyy', 'ixy', 'ixxPSF', 'iyyPSF', 'ixyPSF', 'snr', 'shape_flag']]


In [61]:
plt.hist(src_subselection['iyyPSF'],  bins=np.arange(-0.25,0.5,0.005), label='iyyPSF');
plt.hist(src_subselection['ixxPSF'],  bins=np.arange(-0.25,0.5,0.005), label='ixxPSF');

plt.hist(src_subselection['iyy'],  bins=np.arange(-0.25,0.5,0.005), label='iyy',alpha=0.5);
plt.hist(src_subselection['ixx'],  bins=np.arange(-0.25,0.5,0.005), label='ixx', alpha=0.5);

plt.xlim(-0.25, 0.5)
plt.legend()
plt.xlabel('Shape [nmgy*asec2]')
plt.ylabel('Counts')
plt.title('Shape Histogram of diaSources, all 6 bands, ComCam')